# NGDS Cleaning and Zone Assignment

This script processes raw TRANSCOM data by using various sources to assign zones to the data, then cleans the results.

# Imports & Initial Data Cleaning

In [ ]:
#Imports
import pandas as pd
import numpy as np
from haversine import haversine, Unit
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QVBoxLayout, QHBoxLayout, QPushButton, QComboBox
import os

#Get the absolute path of the folder where this script is located
script_dir = os.path.dirname(os.path.abspath(__file__)) if "__file__" in globals() else os.getcwd()

#Define the input and output directories
input_dir = os.path.join(script_dir, "Input")
output_dir = os.path.join(script_dir, "Output")

#Ensure both directories exist
os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

Due to the collective size of this data, only three months of data can be cleaned and processed at one time.

Raw NGDS data files should be:
1. Located inside the "Input" folder within this "Cleaning Script" folder
2. Formatted as a .txt file
3. Titled using the following format: "NGDS [Month] [Year]" -- (Examples: "NGDS June 2024" or "NGDS April 2023")

In [ ]:
#Get the absolute path of the script's directory
script_dir = os.path.dirname(os.path.abspath(__file__)) if "__file__" in globals() else os.getcwd()

#Define Input and Output directories
input_dir = os.path.join(script_dir, "Input")
output_dir = os.path.join(script_dir, "Output")

#Ensure both directories exist
os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

#Global variable to store the output filename
data = None
output_filename = ""

class MonthYearSelector(QWidget):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("Select Months and Years to Load")
        self.setGeometry(400, 400, 500, 250)
        self.setWindowFlags(self.windowFlags())
        self.activateWindow()
        self.raise_()


        #Available months and years
        self.available_months = [
            "January", "February", "March", "April", "May", "June",
            "July", "August", "September", "October", "November", "December"
        ]
        self.available_years = ["2023", "2024", "2025", "2026", "2027"]

        #Layout
        layout = QVBoxLayout()

        #Month-Year Selection Rows
        self.month_year_selections = []
        for i in range(3):
            row_layout = QHBoxLayout()

            # Month Dropdown
            month_dropdown = QComboBox()
            month_dropdown.addItems([""] + self.available_months)
            month_dropdown.setFixedWidth(150)

            # Year Dropdown
            year_dropdown = QComboBox()
            year_dropdown.addItems([""] + self.available_years)
            year_dropdown.setFixedWidth(80)

            row_layout.addWidget(QLabel(f"Select Month {i+1}:"))
            row_layout.addWidget(month_dropdown)
            row_layout.addWidget(QLabel("Year:"))
            row_layout.addWidget(year_dropdown)

            self.month_year_selections.append((month_dropdown, year_dropdown))
            layout.addLayout(row_layout)

        #Load Data Button
        self.load_button = QPushButton("Load Data")
        self.load_button.clicked.connect(self.load_data)
        layout.addWidget(self.load_button)

        self.setLayout(layout)

    def load_data(self):
        """Loads selected months and years into a DataFrame."""
        global data, output_filename  # Ensure variables are accessible

        selected_files = []
        selected_months_years = []  # Store selected months and years for naming the file

        for month_dropdown, year_dropdown in self.month_year_selections:
            month = month_dropdown.currentText()
            year = year_dropdown.currentText()

            if month and year:
                file_name = f"NGDS {month} {year}.txt"
                file_path = os.path.join(input_dir, file_name)  # Read from Input folder
                selected_files.append(file_path)
                selected_months_years.append((month, year))  # Store for naming

        if not selected_files:
            print("Please select at least one month and year.")
            return

        print(f"Loading data from: {', '.join(selected_files)}...")

        # Load and concatenate selected months
        data_frames = []
        for file_path in selected_files:
            try:
                df = pd.read_csv(file_path, delimiter="|", low_memory=True)
                data_frames.append(df)
            except FileNotFoundError:
                print(f"Warning: {file_path} not found. Skipping...")

        if not data_frames:
            print("No valid data files found. Exiting...")
            return

        data = pd.concat(data_frames, ignore_index=True)

        # Generate the filename dynamically (but don't save yet)
        unique_years = sorted(set(year for _, year in selected_months_years))  # Get unique years
        month_abbr = [month[:3] for month, _ in selected_months_years]  # Convert months to short form
        output_filename = f"ClnNGDS_{'-'.join(month_abbr)}_{'-'.join(unique_years)}.csv"

        print(f"Data successfully loaded. The file will be saved as '{output_filename}' in the Output folder at the end of the script.")

        self.close()  # Close the GUI window once data is loaded

# Launch the GUI
def launch_gui():
    app = QApplication(sys.argv)
    window = MonthYearSelector()
    window.show()
    app.exec_()

# Run the GUI
launch_gui()

In [ ]:
### Data Cleaning and Filtering

#Dropping columns that are not needed
cols_to_drop = [
    'SHIPPING_AGENCY_RPT_LVL', 'SHIPPING_AGENCY_LVL_1','SHIPPING_AGENCY_LVL_2','BILL_OF_LADING',
    'CONTRACT_TYPE','TPPS_DOCUMENT_ID','PURCHASE_ORDER_SHIPMENT_ID','RATE_PROFILE_ID','SERVICE_LVL_MODE',
    'SERVICE_LVL_SPEED','SERVICE_LVL_DLVY_TM','COMMITTED_DELIVERY_DATE', 'SHIPPER_COUNTRY',
    'SHIPPER_COUNTRY_CODE', 'SHIPPER_NGDS_THEATER', 'SHIPPER_NGDS_REGION', 'RECIPIENT_COUNTRY',
    'RECIPIENT_COUNTRY_CODE','RECIPIENT_NGDS_THEATER','RECIPIENT_NGDS_REGION'    
]
data = data.drop(columns = cols_to_drop)

#Removing N/A Service types
data['SERVICE_LVL_CD_DESC'] = data['SERVICE_LVL_CD_DESC'].replace({
    'Home Delivery':'FedEx Home Delivery',
    'Ground':'FedEx Ground'})

#Filtering for only the lower 48 States & DC
state_abbreviations = ['AL', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'ID',
                       'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI',
                       'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY',
                       'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN',
                       'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'DC']
data = data[data['RECIPIENT_STATE_PRVNC'].isin(state_abbreviations)]
data = data[data['SHIPPER_STATE_PRVNC'].isin(state_abbreviations)]

#Converting Pick-up time to Date-time object
data['ACTUAL_PICKUP_DATE'] = pd.to_datetime(data['ACTUAL_PICKUP_DATE'])
data.info()

# Zone Assignments

## Zone Assignments - City Coordinates

Using data from the US Census Bureau, we will match City/State names with coordinates, using those to calculate shipping distance and then shipping zone. 

Then using another dataset, we will match Shipper/Recipient Zip codes for each package.

In [ ]:
#Load and clean places DF
places = pd.read_csv("PopulatedPlaces_National.txt", delimiter = "|")

#Only Columns needed: City, State, Lat, Long
places = places[['feature_name','state_name','prim_lat_dec','prim_long_dec']]

#Rename Columns
rn = {'feature_name':'city','state_name':'state','prim_lat_dec':'lat','prim_long_dec':'long'}
places = places.rename(columns = rn)

#Creating State Abbreviations
places['state'] = places['state'].replace({
    "Alabama": "AL", "Arizona": "AZ", "Arkansas": "AR", "California": "CA",
    "Colorado": "CO", "Connecticut": "CT","Delaware": "DE", "District of Columbia": "DC",
    "Florida": "FL", "Georgia": "GA", "Idaho": "ID", "Illinois": "IL", "Indiana": "IN", "Iowa": "IA",
    "Kansas": "KS", "Kentucky": "KY", "Louisiana": "LA", "Maine": "ME", "Maryland": "MD", "Massachusetts": "MA",
    "Michigan": "MI", "Minnesota": "MN", "Mississippi": "MS", "Missouri": "MO", "Montana": "MT", "Nebraska": "NE",
    "Nevada": "NV", "New Hampshire": "NH", "New Jersey": "NJ", "New Mexico": "NM", "New York": "NY", "North Carolina": "NC",
    "North Dakota": "ND", "Ohio": "OH", "Oklahoma": "OK", "Oregon": "OR", "Pennsylvania": "PA", "Rhode Island": "RI",
    "South Carolina": "SC", "South Dakota": "SD", "Tennessee": "TN", "Texas": "TX", "Utah": "UT", "Vermont": "VT",
    "Virginia": "VA", "Washington": "WA", "West Virginia": "WV", "Wisconsin": "WI", "Wyoming": "WY"
})

#City to UPPER
places['city'] = places['city'].map(lambda x: x.upper())

#Replace Values in Places
replace = {
    'SAINT LOUIS':'ST. LOUIS',
    'SAINT PETERSBURG':'ST. PETERSBURG',
    'CHICAGO LOOP':'CHICAGO',
    'LAKE WORTH BEACH':'LAKE WORTH',
    'PARKCHESTER': 'THE BRONX',
    'LEES SUMMIT':"LEE'S SUMMIT", 
}
places['city'] = places['city'].replace(replace)

#Lower 48 & DC Only
places = places[places['state'].isin(state_abbreviations)]

#Merging Places with TRANSCOM data to get Lats and Longs 
data = data.merge(places, how='left',
                  left_on = ['RECIPIENT_CITY_NAME', 'RECIPIENT_STATE_PRVNC'],
                  right_on = ['city', 'state']).merge(places, 
                                                      how = 'left',
                                                      left_on = ['SHIPPER_CITY_NAME', 'SHIPPER_STATE_PRVNC'],
                                                      right_on = ['city', 'state'],
                                                      suffixes = ['_RECIPIENT','_SHIPPER']).drop(columns=['city_RECIPIENT','city_SHIPPER','state_RECIPIENT','state_SHIPPER'])

#Calculating distances betweem Shipper City & Recipient City
def calculate_distance(df, lat1, lon1, lat2, lon2):
    coords_1 = list(zip(df[lat1], df[lon1]))
    coords_2 = list(zip(df[lat2], df[lon2]))
    distances = [haversine(coord1, coord2, unit=Unit.MILES) for coord1, coord2 in zip(coords_1, coords_2)]
    return pd.Series(distances)
    
data['miles'] = calculate_distance(data, 'lat_RECIPIENT', 'long_RECIPIENT', 'lat_SHIPPER', 'long_SHIPPER')
data = data.drop(columns = ['lat_RECIPIENT', 'long_RECIPIENT', 'lat_SHIPPER', 'long_SHIPPER'])

#Creating Zones based on calculated distances
bins = [-1, 50, 150, 300, 600, 1000, 1400, 1800, float('inf')]
labels = ['Zone 1', 'Zone 2', 'Zone 3', 'Zone 4', 'Zone 5', 'Zone 6', 'Zone 7', 'Zone 8']
data['ZONE'] = pd.cut(data['miles'], bins=bins, labels=labels, right=True)

#Clean up
del places
data = data.drop(columns='miles')

In [ ]:
# ZIP CODE MAPPINGS

#Import USZIPS dataset
usa = pd.read_csv("uszips.csv")
usa = usa[['zip','city','state_id']]
usa = usa.drop_duplicates(subset = ['city','state_id'])
usa['city'] = usa['city'].str.upper()

#Map ZIPs to dataset
data = pd.merge(data, usa, 
                  left_on=['SHIPPER_CITY_NAME','SHIPPER_STATE_PRVNC'],
                 right_on = ['city','state_id'], how = 'left')
data = data.drop(['city','state_id'], axis = 1)
data = data.rename(columns = {'zip':'SHIPPER_ZIP'})

data = pd.merge(data, usa, 
                  left_on=['RECIPIENT_CITY_NAME','RECIPIENT_STATE_PRVNC'],
                 right_on = ['city','state_id'], how = 'left')
data = data.drop(['city','state_id'], axis = 1)
data = data.rename(columns = {'zip':'RECIPIENT_ZIP'})

del usa

In [ ]:
#Filling in some large missing values
data.loc[(data['SHIPPER_CITY_NAME'] == 'ST. LOUIS') & (data['SHIPPER_ZIP'].isna()), 'SHIPPER_ZIP'] = '63101'
data.loc[(data['RECIPIENT_CITY_NAME'] == 'ST. LOUIS') & (data['RECIPIENT_ZIP'].isna()), 'RECIPIENT_ZIP'] = '63101'

data.loc[(data['SHIPPER_CITY_NAME'] == 'URBANCREST') & (data['SHIPPER_ZIP'].isna()), 'SHIPPER_ZIP'] = '43123'
data.loc[(data['RECIPIENT_CITY_NAME'] == 'URBANCREST') & (data['RECIPIENT_ZIP'].isna()), 'RECIPIENT_ZIP'] = '43123'

data.loc[(data['SHIPPER_CITY_NAME'] == 'ST. PETERSBURG') & (data['SHIPPER_ZIP'].isna()), 'SHIPPER_ZIP'] = '33701'
data.loc[(data['RECIPIENT_CITY_NAME'] == 'ST. PETERSBURG') & (data['RECIPIENT_ZIP'].isna()), 'RECIPIENT_ZIP'] = '33701'

data.loc[(data['SHIPPER_CITY_NAME'] == 'THE BRONX') & (data['SHIPPER_ZIP'].isna()), 'SHIPPER_ZIP'] = '10460'
data.loc[(data['RECIPIENT_CITY_NAME'] == 'THE BRONX') & (data['RECIPIENT_ZIP'].isna()), 'RECIPIENT_ZIP'] = '10460'

data.loc[(data['SHIPPER_CITY_NAME'] == 'RIVIERA BEACH') & (data['SHIPPER_ZIP'].isna()), 'SHIPPER_ZIP'] = '33404'
data.loc[(data['RECIPIENT_CITY_NAME'] == 'RIVIERA BEACH') & (data['RECIPIENT_ZIP'].isna()), 'RECIPIENT_ZIP'] = '33404'

data.loc[(data['SHIPPER_CITY_NAME'] == 'LANDOVER') & (data['SHIPPER_ZIP'].isna()), 'SHIPPER_ZIP'] = '20784'
data.loc[(data['RECIPIENT_CITY_NAME'] == 'LANDOVER') & (data['RECIPIENT_ZIP'].isna()), 'RECIPIENT_ZIP'] = '20784'

data.loc[(data['SHIPPER_CITY_NAME'] == 'SOUTHAVEN') & (data['SHIPPER_ZIP'].isna()), 'SHIPPER_ZIP'] = '63101'
data.loc[(data['RECIPIENT_CITY_NAME'] == 'SOUTHAVEN') & (data['RECIPIENT_ZIP'].isna()), 'RECIPIENT_ZIP'] = '63101'

data.loc[(data['SHIPPER_CITY_NAME'] == 'WILKES-BARRE') & (data['SHIPPER_ZIP'].isna()), 'SHIPPER_ZIP'] = '18701'
data.loc[(data['RECIPIENT_CITY_NAME'] == 'WILKES-BARRE') & (data['RECIPIENT_ZIP'].isna()), 'RECIPIENT_ZIP'] = '18701'

data.loc[(data['SHIPPER_CITY_NAME'] == 'MOUNTAIN HOME') & (data['SHIPPER_ZIP'].isna()), 'SHIPPER_ZIP'] = '37684'
data.loc[(data['RECIPIENT_CITY_NAME'] == 'MOUNTAIN HOME') & (data['RECIPIENT_ZIP'].isna()), 'RECIPIENT_ZIP'] = '37684'

data.loc[(data['SHIPPER_CITY_NAME'] == 'COMMERCE') & (data['SHIPPER_ZIP'].isna()), 'SHIPPER_ZIP'] = '90022'
data.loc[(data['RECIPIENT_CITY_NAME'] == 'COMMERCE') & (data['RECIPIENT_ZIP'].isna()), 'RECIPIENT_ZIP'] = '90022'

data.loc[(data['SHIPPER_CITY_NAME'] == 'CITY OF INDUSTRY') & (data['SHIPPER_ZIP'].isna()), 'SHIPPER_ZIP'] = '90601'
data.loc[(data['RECIPIENT_CITY_NAME'] == 'CITY OF INDUSTRY') & (data['RECIPIENT_ZIP'].isna()), 'RECIPIENT_ZIP'] = '90601'

data.loc[(data['SHIPPER_CITY_NAME'] == 'IRWINDALE') & (data['SHIPPER_ZIP'].isna()), 'SHIPPER_ZIP'] = '91006'
data.loc[(data['RECIPIENT_CITY_NAME'] == 'IRWINDALE') & (data['RECIPIENT_ZIP'].isna()), 'RECIPIENT_ZIP'] = '91006'

data.loc[(data['SHIPPER_CITY_NAME'] == 'VERNON') & (data['SHIPPER_ZIP'].isna()), 'SHIPPER_ZIP'] = '90023'
data.loc[(data['RECIPIENT_CITY_NAME'] == 'VERNON') & (data['RECIPIENT_ZIP'].isna()), 'RECIPIENT_ZIP'] = '90023'

data.loc[(data['SHIPPER_CITY_NAME'] == 'EASTVALE') & (data['SHIPPER_ZIP'].isna()), 'SHIPPER_ZIP'] = '91752'
data.loc[(data['RECIPIENT_CITY_NAME'] == 'EASTVALE') & (data['RECIPIENT_ZIP'].isna()), 'RECIPIENT_ZIP'] = '91752'

In [ ]:
#Seperate USPS and Fedex
service_levels = ['UPS Ground', 'UPS Next Day Air', 'UPS 2nd Day Air', 'UPS Next Day Air Saver',
                  'UPS 3 Day Select', 'UPS Next Day Air Early A.M. / UPS Next Day Air Early', 'UPS 2nd Day Air A.M.',
                  'FedEx Home Delivery', 'FedEx Priority Overnight', 'FedEx Standard Overnight', 'FedEx Ground',
                  'FedEx Economy', 'FedEx 2Day', 'FedEx First Overnight', 'FedEx 2Day AM', 'FedEx Express Saver']

data_zoned = data[(data['ZONE'].notnull()) | (~data['SERVICE_LVL_CD_DESC'].isin(service_levels))].copy()
data = data[data['ZONE'].isnull() & (data['SERVICE_LVL_CD_DESC'].isin(service_levels))].drop(columns = ["ZONE"])
upso = data[data['CARRIER_SCAC'] == 'UPSO'].copy()
fedx = data[data['CARRIER_SCAC'] == 'FEDX'].copy()

## Zone Assignments - UPS Rates

In [ ]:
#UPS Ground
upso_ground = upso[upso['SERVICE_LVL_CD_DESC'] == 'UPS Ground'].copy()

#Splitting data by Pick-up dates: Before 10/10/23, 10/10/23-10/10/24, After 10/10/24
date_before = pd.Timestamp('2023-10-10')
date_after = pd.Timestamp('2024-10-10')

upso_ground_fy23 = upso_ground[upso_ground['ACTUAL_PICKUP_DATE'] < date_before].copy()
upso_ground_fy24 = upso_ground[(upso_ground['ACTUAL_PICKUP_DATE'] >= date_before) & (upso_ground['ACTUAL_PICKUP_DATE'] <= date_after)].copy()
upso_ground_fy25 = upso_ground[upso_ground['ACTUAL_PICKUP_DATE'] > date_after].copy()

#Loading Excel Sheets
upso_ground_rates_fy23 = pd.read_excel('upso_costs.xlsx', sheet_name = 'ground_rates_fy23').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")
upso_ground_rates_fy24 = pd.read_excel('upso_costs.xlsx', sheet_name = 'ground_rates_fy24').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")
upso_ground_rates_fy25_ = pd.read_excel('upso_costs.xlsx', sheet_name = 'ground_rates_fy25').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")
upso_ground_rates_fy25a = pd.read_excel('upso_costs.xlsx', sheet_name = 'ground_rates_fy25a').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")
upso_ground_rates_fy25 = pd.concat([upso_ground_rates_fy25_,upso_ground_rates_fy25a], ignore_index = True)

#Merging Sheets based on Pick-up Dates
upso_ground_zoned_fy23 = pd.merge(upso_ground_fy23, upso_ground_rates_fy23, 
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")
upso_ground_zoned_fy24 = pd.merge(upso_ground_fy24, upso_ground_rates_fy24,
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")
upso_ground_zoned_fy25 = pd.merge(upso_ground_fy25, upso_ground_rates_fy25,
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")

upso_ground_zoned_fy = pd.concat([upso_ground_zoned_fy23, upso_ground_zoned_fy24, upso_ground_zoned_fy25], ignore_index = True)

#Seperating data where merge was sucessful and unsucessful
upso_ground_zoned0 = upso_ground_zoned_fy[upso_ground_zoned_fy['ZONE'].notnull()].copy()
upso_ground_missing1 = upso_ground_zoned_fy[upso_ground_zoned_fy['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

#Merging table with missing zones to all zone tables, than concat back to successful zones
upso_ground_retry1 = upso_ground_missing1.merge(upso_ground_rates_fy25, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")
upso_ground_zoned1 = upso_ground_retry1[upso_ground_retry1['ZONE'].notnull()].copy()
upso_ground_missing2 = upso_ground_retry1[upso_ground_retry1['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

upso_ground_retry2 = upso_ground_missing2.merge(upso_ground_rates_fy24, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")
upso_ground_zoned2 = upso_ground_retry2[upso_ground_retry2['ZONE'].notnull()].copy()
upso_ground_missing3 = upso_ground_retry2[upso_ground_retry2['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

upso_ground_retry3 = upso_ground_missing3.merge(upso_ground_rates_fy23, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")

upso_ground_zoned = pd.concat([upso_ground_zoned0, upso_ground_zoned1, upso_ground_zoned2, upso_ground_retry3], ignore_index = True)

#Deleting Uneeded Variables to Release Memory
del upso_ground, upso_ground_fy23, upso_ground_fy24, upso_ground_fy25, upso_ground_rates_fy23, upso_ground_rates_fy24, upso_ground_rates_fy25
del upso_ground_zoned_fy23, upso_ground_zoned_fy24, upso_ground_zoned_fy25, upso_ground_zoned_fy, upso_ground_zoned0, upso_ground_zoned1
del upso_ground_zoned2, upso_ground_retry1, upso_ground_retry2, upso_ground_retry3, upso_ground_missing1, upso_ground_missing2, upso_ground_missing3

print(f'UPS Ground Merge Results')
print(f'Missing Zones: ',upso_ground_zoned["ZONE"].isna().sum())
print(f'Total Rows: ', len(upso_ground_zoned))
print(round(((upso_ground_zoned["ZONE"].isna().sum() / len(upso_ground_zoned))*100),2), f'% Zones Missing')


#Shipper 80401 to Recipient 89101 (appears 507 times) is showing for Zone 5, but should be in Zone 4. 

In [ ]:
#UPS Next Day Air
upso_nda = upso[upso['SERVICE_LVL_CD_DESC'] == 'UPS Next Day Air'].copy()

#Splitting data by Pick-up dates: Before 10/10/23, 10/10/23-10/10/24, After 10/10/24
date_before = pd.Timestamp('2023-10-10')
date_after = pd.Timestamp('2024-10-10')

upso_nda_fy23 = upso_nda[upso_nda['ACTUAL_PICKUP_DATE'] < date_before].copy()
upso_nda_fy24 = upso_nda[(upso_nda['ACTUAL_PICKUP_DATE'] >= date_before) & (upso_nda['ACTUAL_PICKUP_DATE'] <= date_after)].copy()
upso_nda_fy25 = upso_nda[upso_nda['ACTUAL_PICKUP_DATE'] > date_after].copy()

#Loading Excel Sheets
upso_nda_rates_fy23 = pd.read_excel('upso_costs.xlsx', sheet_name = 'nda_rates_fy23').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")
upso_nda_rates_fy24 = pd.read_excel('upso_costs.xlsx', sheet_name = 'nda_rates_fy24').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")
upso_nda_rates_fy25 = pd.read_excel('upso_costs.xlsx', sheet_name = 'nda_rates_fy25').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")

#Merging Sheets based on Pick-up Dates
upso_nda_zoned_fy23 = pd.merge(upso_nda_fy23, upso_nda_rates_fy23, 
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")
upso_nda_zoned_fy24 = pd.merge(upso_nda_fy24, upso_nda_rates_fy24,
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")
upso_nda_zoned_fy25 = pd.merge(upso_nda_fy25, upso_nda_rates_fy25,
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")

upso_nda_zoned_fy = pd.concat([upso_nda_zoned_fy23, upso_nda_zoned_fy24, upso_nda_zoned_fy25], ignore_index = True)

#Seperating data where merge was sucessful and unsucessful
upso_nda_zoned0 = upso_nda_zoned_fy[upso_nda_zoned_fy['ZONE'].notnull()].copy()
upso_nda_missing1 = upso_nda_zoned_fy[upso_nda_zoned_fy['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

#Merging table with missing zones to all zone tables, than concat back to successful zones
upso_nda_retry1 = upso_nda_missing1.merge(upso_nda_rates_fy25, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")
upso_nda_zoned1 = upso_nda_retry1[upso_nda_retry1['ZONE'].notnull()].copy()
upso_nda_missing2 = upso_nda_retry1[upso_nda_retry1['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

upso_nda_retry2 = upso_nda_missing2.merge(upso_nda_rates_fy24, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")
upso_nda_zoned2 = upso_nda_retry2[upso_nda_retry2['ZONE'].notnull()].copy()
upso_nda_missing3 = upso_nda_retry2[upso_nda_retry2['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

upso_nda_retry3 = upso_nda_missing3.merge(upso_nda_rates_fy23, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")

upso_nda_zoned = pd.concat([upso_nda_zoned0, upso_nda_zoned1, upso_nda_zoned2, upso_nda_retry3], ignore_index = True)

#Deleting Uneeded Variables to Release Memory
del upso_nda, upso_nda_fy23, upso_nda_fy24, upso_nda_fy25, upso_nda_rates_fy23, upso_nda_rates_fy24, upso_nda_rates_fy25
del upso_nda_zoned_fy23, upso_nda_zoned_fy24, upso_nda_zoned_fy25, upso_nda_zoned_fy, upso_nda_zoned0, upso_nda_zoned1
del upso_nda_zoned2, upso_nda_retry1, upso_nda_retry2, upso_nda_retry3, upso_nda_missing1, upso_nda_missing2, upso_nda_missing3

print(f'UPS Next Day Air Merge Results')
print(f'Missing Zones: ',upso_nda_zoned["ZONE"].isna().sum())
print(f'Total Rows: ', len(upso_nda_zoned))
print(round(((upso_nda_zoned["ZONE"].isna().sum() / len(upso_nda_zoned))*100),2), f'% Zones Missing')

In [ ]:
#UPS 2nd Day Air
upso_2da = upso[upso['SERVICE_LVL_CD_DESC'] == 'UPS 2nd Day Air'].copy()

#Splitting data by Pick-up dates: Before 10/10/23, 10/10/23-10/10/24, After 10/10/24
date_before = pd.Timestamp('2023-10-10')
date_after = pd.Timestamp('2024-10-10')

upso_2da_fy23 = upso_2da[upso_2da['ACTUAL_PICKUP_DATE'] < date_before].copy()
upso_2da_fy24 = upso_2da[(upso_2da['ACTUAL_PICKUP_DATE'] >= date_before) & (upso_2da['ACTUAL_PICKUP_DATE'] <= date_after)].copy()
upso_2da_fy25 = upso_2da[upso_2da['ACTUAL_PICKUP_DATE'] > date_after].copy()

#Loading Excel Sheets
upso_2da_rates_fy23 = pd.read_excel('upso_costs.xlsx', sheet_name = '2da_rates_fy23').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")
upso_2da_rates_fy24 = pd.read_excel('upso_costs.xlsx', sheet_name = '2da_rates_fy24').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")
upso_2da_rates_fy25 = pd.read_excel('upso_costs.xlsx', sheet_name = '2da_rates_fy25').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")

#Merging Sheets based on Pick-up Dates
upso_2da_zoned_fy23 = pd.merge(upso_2da_fy23, upso_2da_rates_fy23, 
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")
upso_2da_zoned_fy24 = pd.merge(upso_2da_fy24, upso_2da_rates_fy24,
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")
upso_2da_zoned_fy25 = pd.merge(upso_2da_fy25, upso_2da_rates_fy25,
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")

upso_2da_zoned_fy = pd.concat([upso_2da_zoned_fy23, upso_2da_zoned_fy24, upso_2da_zoned_fy25], ignore_index = True)

#Seperating data where merge was sucessful and unsucessful
upso_2da_zoned0 = upso_2da_zoned_fy[upso_2da_zoned_fy['ZONE'].notnull()].copy()
upso_2da_missing1 = upso_2da_zoned_fy[upso_2da_zoned_fy['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

#Merging table with missing zones to all zone tables, than concat back to successful zones
upso_2da_retry1 = upso_2da_missing1.merge(upso_2da_rates_fy25, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")
upso_2da_zoned1 = upso_2da_retry1[upso_2da_retry1['ZONE'].notnull()].copy()
upso_2da_missing2 = upso_2da_retry1[upso_2da_retry1['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

upso_2da_retry2 = upso_2da_missing2.merge(upso_2da_rates_fy24, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")
upso_2da_zoned2 = upso_2da_retry2[upso_2da_retry2['ZONE'].notnull()].copy()
upso_2da_missing3 = upso_2da_retry2[upso_2da_retry2['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

upso_2da_retry3 = upso_2da_missing3.merge(upso_2da_rates_fy23, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")

upso_2da_zoned = pd.concat([upso_2da_zoned0, upso_2da_zoned1, upso_2da_zoned2, upso_2da_retry3], ignore_index = True)

#Deleting Uneeded Variables to Release Memory
del upso_2da, upso_2da_fy23, upso_2da_fy24, upso_2da_fy25, upso_2da_rates_fy23, upso_2da_rates_fy24, upso_2da_rates_fy25
del upso_2da_zoned_fy23, upso_2da_zoned_fy24, upso_2da_zoned_fy25, upso_2da_zoned_fy, upso_2da_zoned0, upso_2da_zoned1
del upso_2da_zoned2, upso_2da_retry1, upso_2da_retry2, upso_2da_retry3, upso_2da_missing1, upso_2da_missing2, upso_2da_missing3

print(f'UPS 2nd Day Air Merge Results')
print(f'Missing Zones: ',upso_2da_zoned["ZONE"].isna().sum())
print(f'Total Rows: ', len(upso_2da_zoned))
print(round(((upso_2da_zoned["ZONE"].isna().sum() / len(upso_2da_zoned))*100),2), f'% Zones Missing')

In [ ]:
#UPS Next Day Air Saver
upso_ndas = upso[upso['SERVICE_LVL_CD_DESC'] == 'UPS Next Day Air Saver'].copy()

#Splitting data by Pick-up dates: Before 10/10/23, 10/10/23-10/10/24, After 10/10/24
date_before = pd.Timestamp('2023-10-10')
date_after = pd.Timestamp('2024-10-10')

upso_ndas_fy23 = upso_ndas[upso_ndas['ACTUAL_PICKUP_DATE'] < date_before].copy()
upso_ndas_fy24 = upso_ndas[(upso_ndas['ACTUAL_PICKUP_DATE'] >= date_before) & (upso_ndas['ACTUAL_PICKUP_DATE'] <= date_after)].copy()
upso_ndas_fy25 = upso_ndas[upso_ndas['ACTUAL_PICKUP_DATE'] > date_after].copy()

#Loading Excel Sheets
upso_ndas_rates_fy23 = pd.read_excel('upso_costs.xlsx', sheet_name = 'ndas_rates_fy23').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")
upso_ndas_rates_fy24 = pd.read_excel('upso_costs.xlsx', sheet_name = 'ndas_rates_fy24').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")
upso_ndas_rates_fy25 = pd.read_excel('upso_costs.xlsx', sheet_name = 'ndas_rates_fy25').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")

#Merging Sheets based on Pick-up Dates
upso_ndas_zoned_fy23 = pd.merge(upso_ndas_fy23, upso_ndas_rates_fy23, 
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")
upso_ndas_zoned_fy24 = pd.merge(upso_ndas_fy24, upso_ndas_rates_fy24,
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")
upso_ndas_zoned_fy25 = pd.merge(upso_ndas_fy25, upso_ndas_rates_fy25,
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")

upso_ndas_zoned_fy = pd.concat([upso_ndas_zoned_fy23, upso_ndas_zoned_fy24, upso_ndas_zoned_fy25], ignore_index = True)

#Seperating data where merge was sucessful and unsucessful
upso_ndas_zoned0 = upso_ndas_zoned_fy[upso_ndas_zoned_fy['ZONE'].notnull()].copy()
upso_ndas_missing1 = upso_ndas_zoned_fy[upso_ndas_zoned_fy['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

#Merging table with missing zones to all zone tables, than concat back to successful zones
upso_ndas_retry1 = upso_ndas_missing1.merge(upso_ndas_rates_fy25, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")
upso_ndas_zoned1 = upso_ndas_retry1[upso_ndas_retry1['ZONE'].notnull()].copy()
upso_ndas_missing2 = upso_ndas_retry1[upso_ndas_retry1['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

upso_ndas_retry2 = upso_ndas_missing2.merge(upso_ndas_rates_fy24, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")
upso_ndas_zoned2 = upso_ndas_retry2[upso_ndas_retry2['ZONE'].notnull()].copy()
upso_ndas_missing3 = upso_ndas_retry2[upso_ndas_retry2['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

upso_ndas_retry3 = upso_ndas_missing3.merge(upso_ndas_rates_fy23, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")

upso_ndas_zoned = pd.concat([upso_ndas_zoned0, upso_ndas_zoned1, upso_ndas_zoned2, upso_ndas_retry3], ignore_index = True)

#Deleting Uneeded Variables to Release Memory
del upso_ndas, upso_ndas_fy23, upso_ndas_fy24, upso_ndas_fy25, upso_ndas_rates_fy23, upso_ndas_rates_fy24, upso_ndas_rates_fy25
del upso_ndas_zoned_fy23, upso_ndas_zoned_fy24, upso_ndas_zoned_fy25, upso_ndas_zoned_fy, upso_ndas_zoned0, upso_ndas_zoned1
del upso_ndas_zoned2, upso_ndas_retry1, upso_ndas_retry2, upso_ndas_retry3, upso_ndas_missing1, upso_ndas_missing2, upso_ndas_missing3

print(f'UPS Next Day Air Saver Merge Results')
print(f'Missing Zones: ',upso_ndas_zoned["ZONE"].isna().sum())
print(f'Total Rows: ', len(upso_ndas_zoned))
print(round(((upso_ndas_zoned["ZONE"].isna().sum() / len(upso_ndas_zoned))*100),2), f'% Zones Missing')

In [ ]:
#UPS 3 Day Select
upso_3ds = upso[upso['SERVICE_LVL_CD_DESC'] == 'UPS 3 Day Select'].copy()

#Splitting data by Pick-up dates: Before 10/10/23, 10/10/23-10/10/24, After 10/10/24
date_before = pd.Timestamp('2023-10-10')
date_after = pd.Timestamp('2024-10-10')

upso_3ds_fy23 = upso_3ds[upso_3ds['ACTUAL_PICKUP_DATE'] < date_before].copy()
upso_3ds_fy24 = upso_3ds[(upso_3ds['ACTUAL_PICKUP_DATE'] >= date_before) & (upso_3ds['ACTUAL_PICKUP_DATE'] <= date_after)].copy()
upso_3ds_fy25 = upso_3ds[upso_3ds['ACTUAL_PICKUP_DATE'] > date_after].copy()

#Loading Excel Sheets
upso_3ds_rates_fy23 = pd.read_excel('upso_costs.xlsx', sheet_name = '3ds_rates_fy23').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")
upso_3ds_rates_fy24 = pd.read_excel('upso_costs.xlsx', sheet_name = '3ds_rates_fy24').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")
upso_3ds_rates_fy25 = pd.read_excel('upso_costs.xlsx', sheet_name = '3ds_rates_fy25').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")

#Merging Sheets based on Pick-up Dates
upso_3ds_zoned_fy23 = pd.merge(upso_3ds_fy23, upso_3ds_rates_fy23, 
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")
upso_3ds_zoned_fy24 = pd.merge(upso_3ds_fy24, upso_3ds_rates_fy24,
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")
upso_3ds_zoned_fy25 = pd.merge(upso_3ds_fy25, upso_3ds_rates_fy25,
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")

upso_3ds_zoned_fy = pd.concat([upso_3ds_zoned_fy23, upso_3ds_zoned_fy24, upso_3ds_zoned_fy25], ignore_index = True)

#Seperating data where merge was sucessful and unsucessful
upso_3ds_zoned0 = upso_3ds_zoned_fy[upso_3ds_zoned_fy['ZONE'].notnull()].copy()
upso_3ds_missing1 = upso_3ds_zoned_fy[upso_3ds_zoned_fy['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

#Merging table with missing zones to all zone tables, than concat back to successful zones
upso_3ds_retry1 = upso_3ds_missing1.merge(upso_3ds_rates_fy25, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")
upso_3ds_zoned1 = upso_3ds_retry1[upso_3ds_retry1['ZONE'].notnull()].copy()
upso_3ds_missing2 = upso_3ds_retry1[upso_3ds_retry1['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

upso_3ds_retry2 = upso_3ds_missing2.merge(upso_3ds_rates_fy24, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")
upso_3ds_zoned2 = upso_3ds_retry2[upso_3ds_retry2['ZONE'].notnull()].copy()
upso_3ds_missing3 = upso_3ds_retry2[upso_3ds_retry2['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

upso_3ds_retry3 = upso_3ds_missing3.merge(upso_3ds_rates_fy23, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")

upso_3ds_zoned = pd.concat([upso_3ds_zoned0, upso_3ds_zoned1, upso_3ds_zoned2, upso_3ds_retry3], ignore_index = True)

#Deleting Uneeded Variables to Release Memory
del upso_3ds, upso_3ds_fy23, upso_3ds_fy24, upso_3ds_fy25, upso_3ds_rates_fy23, upso_3ds_rates_fy24, upso_3ds_rates_fy25
del upso_3ds_zoned_fy23, upso_3ds_zoned_fy24, upso_3ds_zoned_fy25, upso_3ds_zoned_fy, upso_3ds_zoned0, upso_3ds_zoned1
del upso_3ds_zoned2, upso_3ds_retry1, upso_3ds_retry2, upso_3ds_retry3, upso_3ds_missing1, upso_3ds_missing2, upso_3ds_missing3

print(f'UPS 3 Day Select Merge Results')
print(f'Missing Zones: ',upso_3ds_zoned["ZONE"].isna().sum())
print(f'Total Rows: ', len(upso_3ds_zoned))
print(round(((upso_3ds_zoned["ZONE"].isna().sum() / len(upso_3ds_zoned))*100),2), f'% Zones Missing')

In [ ]:
#UPS Next Day Air Early A.M. / UPS Next Day Air Early
upso_ndae = upso[upso['SERVICE_LVL_CD_DESC'] == 'UPS Next Day Air Early A.M. / UPS Next Day Air Early'].copy()

#Splitting data by Pick-up dates: Before 10/10/23, 10/10/23-10/10/24, After 10/10/24
date_before = pd.Timestamp('2023-10-10')
date_after = pd.Timestamp('2024-10-10')

upso_ndae_fy23 = upso_ndae[upso_ndae['ACTUAL_PICKUP_DATE'] < date_before].copy()
upso_ndae_fy24 = upso_ndae[(upso_ndae['ACTUAL_PICKUP_DATE'] >= date_before) & (upso_ndae['ACTUAL_PICKUP_DATE'] <= date_after)].copy()
upso_ndae_fy25 = upso_ndae[upso_ndae['ACTUAL_PICKUP_DATE'] > date_after].copy()

#Loading Excel Sheets
upso_ndae_rates_fy23 = pd.read_excel('upso_costs.xlsx', sheet_name = 'ndae_rates_fy23').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")
upso_ndae_rates_fy24 = pd.read_excel('upso_costs.xlsx', sheet_name = 'ndae_rates_fy24').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")
upso_ndae_rates_fy25 = pd.read_excel('upso_costs.xlsx', sheet_name = 'ndae_rates_fy25').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")

#Merging Sheets based on Pick-up Dates
upso_ndae_zoned_fy23 = pd.merge(upso_ndae_fy23, upso_ndae_rates_fy23, 
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")
upso_ndae_zoned_fy24 = pd.merge(upso_ndae_fy24, upso_ndae_rates_fy24,
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")
upso_ndae_zoned_fy25 = pd.merge(upso_ndae_fy25, upso_ndae_rates_fy25,
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")

upso_ndae_zoned_fy = pd.concat([upso_ndae_zoned_fy23, upso_ndae_zoned_fy24, upso_ndae_zoned_fy25], ignore_index = True)

#Seperating data where merge was sucessful and unsucessful
upso_ndae_zoned0 = upso_ndae_zoned_fy[upso_ndae_zoned_fy['ZONE'].notnull()].copy()
upso_ndae_missing1 = upso_ndae_zoned_fy[upso_ndae_zoned_fy['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

#Merging table with missing zones to all zone tables, than concat back to successful zones
upso_ndae_retry1 = upso_ndae_missing1.merge(upso_ndae_rates_fy25, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")
upso_ndae_zoned1 = upso_ndae_retry1[upso_ndae_retry1['ZONE'].notnull()].copy()
upso_ndae_missing2 = upso_ndae_retry1[upso_ndae_retry1['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

upso_ndae_retry2 = upso_ndae_missing2.merge(upso_ndae_rates_fy24, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")
upso_ndae_zoned2 = upso_ndae_retry2[upso_ndae_retry2['ZONE'].notnull()].copy()
upso_ndae_missing3 = upso_ndae_retry2[upso_ndae_retry2['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

upso_ndae_retry3 = upso_ndae_missing3.merge(upso_ndae_rates_fy23, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")

upso_ndae_zoned = pd.concat([upso_ndae_zoned0, upso_ndae_zoned1, upso_ndae_zoned2, upso_ndae_retry3], ignore_index = True)

#Deleting Uneeded Variables to Release Memory
del upso_ndae, upso_ndae_fy23, upso_ndae_fy24, upso_ndae_fy25, upso_ndae_rates_fy23, upso_ndae_rates_fy24, upso_ndae_rates_fy25
del upso_ndae_zoned_fy23, upso_ndae_zoned_fy24, upso_ndae_zoned_fy25, upso_ndae_zoned_fy, upso_ndae_zoned0, upso_ndae_zoned1
del upso_ndae_zoned2, upso_ndae_retry1, upso_ndae_retry2, upso_ndae_retry3, upso_ndae_missing1, upso_ndae_missing2, upso_ndae_missing3

print(f'UPS Next Day Air Early A.M. / UPS Next Day Air Early Merge Results')
print(f'Missing Zones: ',upso_ndae_zoned["ZONE"].isna().sum())
print(f'Total Rows: ', len(upso_ndae_zoned))
print(round(((upso_ndae_zoned["ZONE"].isna().sum() / len(upso_ndae_zoned))*100),2), f'% Zones Missing')

In [ ]:
#UPS 2nd Day Air A.M.
upso_2daam = upso[upso['SERVICE_LVL_CD_DESC'] == 'UPS 2nd Day Air A.M.'].copy()

#Splitting data by Pick-up dates: Before 10/10/23, 10/10/23-10/10/24, After 10/10/24
date_before = pd.Timestamp('2023-10-10')
date_after = pd.Timestamp('2024-10-10')

upso_2daam_fy23 = upso_2daam[upso_2daam['ACTUAL_PICKUP_DATE'] < date_before].copy()
upso_2daam_fy24 = upso_2daam[(upso_2daam['ACTUAL_PICKUP_DATE'] >= date_before) & (upso_2daam['ACTUAL_PICKUP_DATE'] <= date_after)].copy()
upso_2daam_fy25 = upso_2daam[upso_2daam['ACTUAL_PICKUP_DATE'] > date_after].copy()

#Loading Excel Sheets
upso_2daam_rates_fy23 = pd.read_excel('upso_costs.xlsx', sheet_name = '2daam_rates_fy23').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")
upso_2daam_rates_fy24 = pd.read_excel('upso_costs.xlsx', sheet_name = '2daam_rates_fy24').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")
upso_2daam_rates_fy25 = pd.read_excel('upso_costs.xlsx', sheet_name = '2daam_rates_fy25').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")

#Merging Sheets based on Pick-up Dates
upso_2daam_zoned_fy23 = pd.merge(upso_2daam_fy23, upso_2daam_rates_fy23, 
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")
upso_2daam_zoned_fy24 = pd.merge(upso_2daam_fy24, upso_2daam_rates_fy24,
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")
upso_2daam_zoned_fy25 = pd.merge(upso_2daam_fy25, upso_2daam_rates_fy25,
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")

upso_2daam_zoned_fy = pd.concat([upso_2daam_zoned_fy23, upso_2daam_zoned_fy24, upso_2daam_zoned_fy25], ignore_index = True)

#Seperating data where merge was sucessful and unsucessful
upso_2daam_zoned0 = upso_2daam_zoned_fy[upso_2daam_zoned_fy['ZONE'].notnull()].copy()
upso_2daam_missing1 = upso_2daam_zoned_fy[upso_2daam_zoned_fy['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

#Merging table with missing zones to all zone tables, than concat back to successful zones
upso_2daam_retry1 = upso_2daam_missing1.merge(upso_2daam_rates_fy25, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")
upso_2daam_zoned1 = upso_2daam_retry1[upso_2daam_retry1['ZONE'].notnull()].copy()
upso_2daam_missing2 = upso_2daam_retry1[upso_2daam_retry1['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

upso_2daam_retry2 = upso_2daam_missing2.merge(upso_2daam_rates_fy24, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")
upso_2daam_zoned2 = upso_2daam_retry2[upso_2daam_retry2['ZONE'].notnull()].copy()
upso_2daam_missing3 = upso_2daam_retry2[upso_2daam_retry2['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

upso_2daam_retry3 = upso_2daam_missing3.merge(upso_2daam_rates_fy23, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")

upso_2daam_zoned = pd.concat([upso_2daam_zoned0, upso_2daam_zoned1, upso_2daam_zoned2, upso_2daam_retry3], ignore_index = True)

#Deleting Uneeded Variables to Release Memory
del upso_2daam, upso_2daam_fy23, upso_2daam_fy24, upso_2daam_fy25, upso_2daam_rates_fy23, upso_2daam_rates_fy24, upso_2daam_rates_fy25
del upso_2daam_zoned_fy23, upso_2daam_zoned_fy24, upso_2daam_zoned_fy25, upso_2daam_zoned_fy, upso_2daam_zoned0, upso_2daam_zoned1
del upso_2daam_zoned2, upso_2daam_retry1, upso_2daam_retry2, upso_2daam_retry3, upso_2daam_missing1, upso_2daam_missing2, upso_2daam_missing3

print(f'UPS 2nd Day Air A.M. Merge Results')
print(f'Missing Zones: ',upso_2daam_zoned["ZONE"].isna().sum())
print(f'Total Rows: ', len(upso_2daam_zoned))
print(round(((upso_2daam_zoned["ZONE"].isna().sum() / len(upso_2daam_zoned))*100),2), f'% Zones Missing')

In [ ]:
#Append all UPS
upso_dfs = [upso_ground_zoned, upso_nda_zoned, upso_2da_zoned, upso_ndas_zoned, upso_3ds_zoned, upso_ndae_zoned, upso_2daam_zoned]
upso_zoned = pd.concat(upso_dfs, ignore_index = True)

del upso_ground_zoned, upso_nda_zoned, upso_2da_zoned, upso_ndas_zoned, upso_3ds_zoned, upso_ndae_zoned, upso_2daam_zoned

print(f'UPS Merge Results')
print(f'Missing Zones: ',upso_zoned["ZONE"].isna().sum())
print(f'Total Rows: ', len(upso_zoned))
print(round(((upso_zoned["ZONE"].isna().sum() / len(upso_zoned))*100),2), f'% Zones Missing')

## Zone Assignments - FEDEX Rates

In [ ]:
#FEDEX Home Delivery
fedx_hd = fedx[fedx['SERVICE_LVL_CD_DESC'] == 'FedEx Home Delivery'].copy()

#Splitting data by Pick-up dates: Before 10/10/23, 10/10/23-10/10/24, After 10/10/24
date_before = pd.Timestamp('2023-10-10')
date_after = pd.Timestamp('2024-10-10')

fedx_hd_fy23 = fedx_hd[fedx_hd['ACTUAL_PICKUP_DATE'] < date_before].copy()
fedx_hd_fy24 = fedx_hd[(fedx_hd['ACTUAL_PICKUP_DATE'] >= date_before) & (fedx_hd['ACTUAL_PICKUP_DATE'] <= date_after)].copy()
fedx_hd_fy25 = fedx_hd[fedx_hd['ACTUAL_PICKUP_DATE'] > date_after].copy()

#Loading Excel Sheets
fedx_hd_rates_fy23 = pd.read_excel('fedex_costs.xlsx', sheet_name = 'hd_rates_fy23').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")
fedx_hd_rates_fy24 = pd.read_excel('fedex_costs.xlsx', sheet_name = 'hd_rates_fy24').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")
fedx_hd_rates_fy25 = pd.read_excel('fedex_costs.xlsx', sheet_name = 'hd_rates_fy25').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")

#Merging Sheets based on Pick-up Dates
fedx_hd_zoned_fy23 = pd.merge(fedx_hd_fy23, fedx_hd_rates_fy23, 
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")
fedx_hd_zoned_fy24 = pd.merge(fedx_hd_fy24, fedx_hd_rates_fy24,
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")
fedx_hd_zoned_fy25 = pd.merge(fedx_hd_fy25, fedx_hd_rates_fy25,
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")

fedx_hd_zoned_fy = pd.concat([fedx_hd_zoned_fy23, fedx_hd_zoned_fy24, fedx_hd_zoned_fy25], ignore_index = True)

#Seperating data where merge was sucessful and unsucessful
fedx_hd_zoned0 = fedx_hd_zoned_fy[fedx_hd_zoned_fy['ZONE'].notnull()].copy()
fedx_hd_missing1 = fedx_hd_zoned_fy[fedx_hd_zoned_fy['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

#Merging table with missing zones to all zone tables, than concat back to successful zones
fedx_hd_retry1 = fedx_hd_missing1.merge(fedx_hd_rates_fy25, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")
fedx_hd_zoned1 = fedx_hd_retry1[fedx_hd_retry1['ZONE'].notnull()].copy()
fedx_hd_missing2 = fedx_hd_retry1[fedx_hd_retry1['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

fedx_hd_retry2 = fedx_hd_missing2.merge(fedx_hd_rates_fy24, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")
fedx_hd_zoned2 = fedx_hd_retry2[fedx_hd_retry2['ZONE'].notnull()].copy()
fedx_hd_missing3 = fedx_hd_retry2[fedx_hd_retry2['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

fedx_hd_retry3 = fedx_hd_missing3.merge(fedx_hd_rates_fy23, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")

fedx_hd_zoned = pd.concat([fedx_hd_zoned0, fedx_hd_zoned1, fedx_hd_zoned2, fedx_hd_retry3], ignore_index = True)

#Deleting Uneeded Variables to Release Memory
del fedx_hd, fedx_hd_fy23, fedx_hd_fy24, fedx_hd_fy25, fedx_hd_rates_fy23, fedx_hd_rates_fy24, fedx_hd_rates_fy25
del fedx_hd_zoned_fy23, fedx_hd_zoned_fy24, fedx_hd_zoned_fy25, fedx_hd_zoned_fy, fedx_hd_zoned0, fedx_hd_zoned1
del fedx_hd_zoned2, fedx_hd_retry1, fedx_hd_retry2, fedx_hd_retry3, fedx_hd_missing1, fedx_hd_missing2, fedx_hd_missing3

print(f'FEDEX Home Delivery Merge Results')
print(f'Missing Zones: ',fedx_hd_zoned["ZONE"].isna().sum())
print(f'Total Rows: ', len(fedx_hd_zoned))
print(round(((fedx_hd_zoned["ZONE"].isna().sum() / len(fedx_hd_zoned))*100),2), f'% Zones Missing')

In [ ]:
#FEDEX Priority Overnight
fedx_po = fedx[fedx['SERVICE_LVL_CD_DESC'] == 'FedEx Priority Overnight'].copy()

#Splitting data by Pick-up dates: Before 10/10/23, 10/10/23-10/10/24, After 10/10/24
date_before = pd.Timestamp('2023-10-10')
date_after = pd.Timestamp('2024-10-10')

fedx_po_fy23 = fedx_po[fedx_po['ACTUAL_PICKUP_DATE'] < date_before].copy()
fedx_po_fy24 = fedx_po[(fedx_po['ACTUAL_PICKUP_DATE'] >= date_before) & (fedx_po['ACTUAL_PICKUP_DATE'] <= date_after)].copy()
fedx_po_fy25 = fedx_po[fedx_po['ACTUAL_PICKUP_DATE'] > date_after].copy()

#Loading Excel Sheets
fedx_po_rates_fy23 = pd.read_excel('fedex_costs.xlsx', sheet_name = 'po_rates_fy23').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")
fedx_po_rates_fy24 = pd.read_excel('fedex_costs.xlsx', sheet_name = 'po_rates_fy24').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")
fedx_po_rates_fy25 = pd.read_excel('fedex_costs.xlsx', sheet_name = 'po_rates_fy25').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")

fedx_po['BILLED_WEIGHT'] = fedx_po['BILLED_WEIGHT'].astype('float')
fedx_po['TOTAL_FREIGHT_AMOUNT'] = fedx_po['TOTAL_FREIGHT_AMOUNT'].astype('float')
fedx_po_rates_fy23['FreightPrice'] = fedx_po_rates_fy23['FreightPrice'].astype('float')
fedx_po_rates_fy23['WEIGHT'] = fedx_po_rates_fy23['WEIGHT'].astype('float32')
fedx_po_rates_fy24['FreightPrice'] = fedx_po_rates_fy24['FreightPrice'].astype('float')
fedx_po_rates_fy24['WEIGHT'] = fedx_po_rates_fy24['WEIGHT'].astype('float')
fedx_po_rates_fy25['FreightPrice'] = fedx_po_rates_fy25['FreightPrice'].astype('float')
fedx_po_rates_fy25['WEIGHT'] = fedx_po_rates_fy25['WEIGHT'].astype('float')

#Merging Sheets based on Pick-up Dates
fedx_po_zoned_fy23 = pd.merge(fedx_po_fy23, fedx_po_rates_fy23, 
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")
fedx_po_zoned_fy24 = pd.merge(fedx_po_fy24, fedx_po_rates_fy24,
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")
fedx_po_zoned_fy25 = pd.merge(fedx_po_fy25, fedx_po_rates_fy25,
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")

fedx_po_zoned_fy = pd.concat([fedx_po_zoned_fy23, fedx_po_zoned_fy24, fedx_po_zoned_fy25], ignore_index = True)

#Seperating data where merge was sucessful and unsucessful
fedx_po_zoned0 = fedx_po_zoned_fy[fedx_po_zoned_fy['ZONE'].notnull()].copy()
fedx_po_missing1 = fedx_po_zoned_fy[fedx_po_zoned_fy['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

#Merging table with missing zones to all zone tables, than concat back to successful zones
fedx_po_retry1 = fedx_po_missing1.merge(fedx_po_rates_fy25, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")
fedx_po_zoned1 = fedx_po_retry1[fedx_po_retry1['ZONE'].notnull()].copy()
fedx_po_missing2 = fedx_po_retry1[fedx_po_retry1['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

fedx_po_retry2 = fedx_po_missing2.merge(fedx_po_rates_fy24, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")
fedx_po_zoned2 = fedx_po_retry2[fedx_po_retry2['ZONE'].notnull()].copy()
fedx_po_missing3 = fedx_po_retry2[fedx_po_retry2['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

fedx_po_retry3 = fedx_po_missing3.merge(fedx_po_rates_fy23, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")

fedx_po_zoned = pd.concat([fedx_po_zoned0, fedx_po_zoned1, fedx_po_zoned2, fedx_po_retry3], ignore_index = True)

#Deleting Uneeded Variables to Release Memory
del fedx_po, fedx_po_fy23, fedx_po_fy24, fedx_po_fy25, fedx_po_rates_fy23, fedx_po_rates_fy24, fedx_po_rates_fy25
del fedx_po_zoned_fy23, fedx_po_zoned_fy24, fedx_po_zoned_fy25, fedx_po_zoned_fy, fedx_po_zoned0, fedx_po_zoned1
del fedx_po_zoned2, fedx_po_retry1, fedx_po_retry2, fedx_po_retry3, fedx_po_missing1, fedx_po_missing2, fedx_po_missing3

print(f'FEDEX Priority Overnight Merge Results')
print(f'Missing Zones: ',fedx_po_zoned["ZONE"].isna().sum())
print(f'Total Rows: ', len(fedx_po_zoned))
print(round(((fedx_po_zoned["ZONE"].isna().sum() / len(fedx_po_zoned))*100),2), f'% Zones Missing')

In [ ]:
#FEDEX Standard Overnight
fedx_so = fedx[fedx['SERVICE_LVL_CD_DESC'] == 'FedEx Standard Overnight'].copy()

#Splitting data by Pick-up dates: Before 10/10/23, 10/10/23-10/10/24, After 10/10/24
date_before = pd.Timestamp('2023-10-10')
date_after = pd.Timestamp('2024-10-10')

fedx_so_fy23 = fedx_so[fedx_so['ACTUAL_PICKUP_DATE'] < date_before].copy()
fedx_so_fy24 = fedx_so[(fedx_so['ACTUAL_PICKUP_DATE'] >= date_before) & (fedx_so['ACTUAL_PICKUP_DATE'] <= date_after)].copy()
fedx_so_fy25 = fedx_so[fedx_so['ACTUAL_PICKUP_DATE'] > date_after].copy()

#Loading Excel Sheets
fedx_so_rates_fy23 = pd.read_excel('fedex_costs.xlsx', sheet_name = 'so_rates_fy23').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")
fedx_so_rates_fy24 = pd.read_excel('fedex_costs.xlsx', sheet_name = 'so_rates_fy24').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")
fedx_so_rates_fy25 = pd.read_excel('fedex_costs.xlsx', sheet_name = 'so_rates_fy25').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")

fedx_so['BILLED_WEIGHT'] = fedx_so['BILLED_WEIGHT'].astype('float')
fedx_so['TOTAL_FREIGHT_AMOUNT'] = fedx_so['TOTAL_FREIGHT_AMOUNT'].astype('float')
fedx_so_rates_fy23['FreightPrice'] = fedx_so_rates_fy23['FreightPrice'].astype('float')
fedx_so_rates_fy23['WEIGHT'] = fedx_so_rates_fy23['WEIGHT'].astype('float32')
fedx_so_rates_fy24['FreightPrice'] = fedx_so_rates_fy24['FreightPrice'].astype('float')
fedx_so_rates_fy24['WEIGHT'] = fedx_so_rates_fy24['WEIGHT'].astype('float')
fedx_so_rates_fy25['FreightPrice'] = fedx_so_rates_fy25['FreightPrice'].astype('float')
fedx_so_rates_fy25['WEIGHT'] = fedx_so_rates_fy25['WEIGHT'].astype('float')

#Merging Sheets based on Pick-up Dates
fedx_so_zoned_fy23 = pd.merge(fedx_so_fy23, fedx_so_rates_fy23, 
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")
fedx_so_zoned_fy24 = pd.merge(fedx_so_fy24, fedx_so_rates_fy24,
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")
fedx_so_zoned_fy25 = pd.merge(fedx_so_fy25, fedx_so_rates_fy25,
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")

fedx_so_zoned_fy = pd.concat([fedx_so_zoned_fy23, fedx_so_zoned_fy24, fedx_so_zoned_fy25], ignore_index = True)

#Seperating data where merge was sucessful and unsucessful
fedx_so_zoned0 = fedx_so_zoned_fy[fedx_so_zoned_fy['ZONE'].notnull()].copy()
fedx_so_missing1 = fedx_so_zoned_fy[fedx_so_zoned_fy['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

#Merging table with missing zones to all zone tables, than concat back to successful zones
fedx_so_retry1 = fedx_so_missing1.merge(fedx_so_rates_fy25, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")
fedx_so_zoned1 = fedx_so_retry1[fedx_so_retry1['ZONE'].notnull()].copy()
fedx_so_missing2 = fedx_so_retry1[fedx_so_retry1['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

fedx_so_retry2 = fedx_so_missing2.merge(fedx_so_rates_fy24, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")
fedx_so_zoned2 = fedx_so_retry2[fedx_so_retry2['ZONE'].notnull()].copy()
fedx_so_missing3 = fedx_so_retry2[fedx_so_retry2['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

fedx_so_retry3 = fedx_so_missing3.merge(fedx_so_rates_fy23, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")

fedx_so_zoned = pd.concat([fedx_so_zoned0, fedx_so_zoned1, fedx_so_zoned2, fedx_so_retry3], ignore_index = True)

#Deleting Uneeded Variables to Release Memory
del fedx_so, fedx_so_fy23, fedx_so_fy24, fedx_so_fy25, fedx_so_rates_fy23, fedx_so_rates_fy24, fedx_so_rates_fy25
del fedx_so_zoned_fy23, fedx_so_zoned_fy24, fedx_so_zoned_fy25, fedx_so_zoned_fy, fedx_so_zoned0, fedx_so_zoned1
del fedx_so_zoned2, fedx_so_retry1, fedx_so_retry2, fedx_so_retry3, fedx_so_missing1, fedx_so_missing2, fedx_so_missing3

print(f'FEDEX Standard Overnight Merge Results')
print(f'Missing Zones: ',fedx_so_zoned["ZONE"].isna().sum())
print(f'Total Rows: ', len(fedx_so_zoned))
print(round(((fedx_so_zoned["ZONE"].isna().sum() / len(fedx_so_zoned))*100),2), f'% Zones Missing')

In [ ]:
#FEDEX Ground
fedx_ground = fedx[fedx['SERVICE_LVL_CD_DESC'] == 'FedEx Ground'].copy()

#Splitting data by Pick-up dates: Before 10/10/23, 10/10/23-10/10/24, After 10/10/24
date_before = pd.Timestamp('2023-10-10')
date_after = pd.Timestamp('2024-10-10')

fedx_ground_fy23 = fedx_ground[fedx_ground['ACTUAL_PICKUP_DATE'] < date_before].copy()
fedx_ground_fy24 = fedx_ground[(fedx_ground['ACTUAL_PICKUP_DATE'] >= date_before) & (fedx_ground['ACTUAL_PICKUP_DATE'] <= date_after)].copy()
fedx_ground_fy25 = fedx_ground[fedx_ground['ACTUAL_PICKUP_DATE'] > date_after].copy()

#Loading Excel Sheets
fedx_ground_rates_fy23 = pd.read_excel('fedex_costs.xlsx', sheet_name = 'ground_rates_fy23').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")
fedx_ground_rates_fy24 = pd.read_excel('fedex_costs.xlsx', sheet_name = 'ground_rates_fy24').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")
fedx_ground_rates_fy25 = pd.read_excel('fedex_costs.xlsx', sheet_name = 'ground_rates_fy25').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")

fedx_ground['BILLED_WEIGHT'] = fedx_ground['BILLED_WEIGHT'].astype('float')
fedx_ground['TOTAL_FREIGHT_AMOUNT'] = fedx_ground['TOTAL_FREIGHT_AMOUNT'].astype('float')
fedx_ground_rates_fy23['FreightPrice'] = fedx_ground_rates_fy23['FreightPrice'].astype('float')
fedx_ground_rates_fy23['WEIGHT'] = fedx_ground_rates_fy23['WEIGHT'].astype('float32')
fedx_ground_rates_fy24['FreightPrice'] = fedx_ground_rates_fy24['FreightPrice'].astype('float')
fedx_ground_rates_fy24['WEIGHT'] = fedx_ground_rates_fy24['WEIGHT'].astype('float')
fedx_ground_rates_fy25['FreightPrice'] = fedx_ground_rates_fy25['FreightPrice'].astype('float')
fedx_ground_rates_fy25['WEIGHT'] = fedx_ground_rates_fy25['WEIGHT'].astype('float')

#Merging Sheets based on Pick-up Dates
fedx_ground_zoned_fy23 = pd.merge(fedx_ground_fy23, fedx_ground_rates_fy23, 
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")
fedx_ground_zoned_fy24 = pd.merge(fedx_ground_fy24, fedx_ground_rates_fy24,
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")
fedx_ground_zoned_fy25 = pd.merge(fedx_ground_fy25, fedx_ground_rates_fy25,
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")

fedx_ground_zoned_fy = pd.concat([fedx_ground_zoned_fy23, fedx_ground_zoned_fy24, fedx_ground_zoned_fy25], ignore_index = True)

#Seperating data where merge was sucessful and unsucessful
fedx_ground_zoned0 = fedx_ground_zoned_fy[fedx_ground_zoned_fy['ZONE'].notnull()].copy()
fedx_ground_missing1 = fedx_ground_zoned_fy[fedx_ground_zoned_fy['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

#Merging table with missing zones to all zone tables, than concat back to successful zones
fedx_ground_retry1 = fedx_ground_missing1.merge(fedx_ground_rates_fy25, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")
fedx_ground_zoned1 = fedx_ground_retry1[fedx_ground_retry1['ZONE'].notnull()].copy()
fedx_ground_missing2 = fedx_ground_retry1[fedx_ground_retry1['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

fedx_ground_retry2 = fedx_ground_missing2.merge(fedx_ground_rates_fy24, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")
fedx_ground_zoned2 = fedx_ground_retry2[fedx_ground_retry2['ZONE'].notnull()].copy()
fedx_ground_missing3 = fedx_ground_retry2[fedx_ground_retry2['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

fedx_ground_retry3 = fedx_ground_missing3.merge(fedx_ground_rates_fy23, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")

fedx_ground_zoned = pd.concat([fedx_ground_zoned0, fedx_ground_zoned1, fedx_ground_zoned2, fedx_ground_retry3], ignore_index = True)

#Deleting Uneeded Variables to Release Memory
del fedx_ground, fedx_ground_fy23, fedx_ground_fy24, fedx_ground_fy25, fedx_ground_rates_fy23, fedx_ground_rates_fy24, fedx_ground_rates_fy25
del fedx_ground_zoned_fy23, fedx_ground_zoned_fy24, fedx_ground_zoned_fy25, fedx_ground_zoned_fy, fedx_ground_zoned0, fedx_ground_zoned1
del fedx_ground_zoned2, fedx_ground_retry1, fedx_ground_retry2, fedx_ground_retry3, fedx_ground_missing1, fedx_ground_missing2, fedx_ground_missing3

print(f'FEDEX Ground Merge Results')
print(f'Missing Zones: ',fedx_ground_zoned["ZONE"].isna().sum())
print(f'Total Rows: ', len(fedx_ground_zoned))
print(round(((fedx_ground_zoned["ZONE"].isna().sum() / len(fedx_ground_zoned))*100),2), f'% Zones Missing')

In [ ]:
#FEDEX 2Day
fedx_2d = fedx[fedx['SERVICE_LVL_CD_DESC'] == 'FedEx 2Day'].copy()

#Splitting data by Pick-up dates: Before 10/10/23, 10/10/23-10/10/24, After 10/10/24
date_before = pd.Timestamp('2023-10-10')
date_after = pd.Timestamp('2024-10-10')

fedx_2d_fy23 = fedx_2d[fedx_2d['ACTUAL_PICKUP_DATE'] < date_before].copy()
fedx_2d_fy24 = fedx_2d[(fedx_2d['ACTUAL_PICKUP_DATE'] >= date_before) & (fedx_2d['ACTUAL_PICKUP_DATE'] <= date_after)].copy()
fedx_2d_fy25 = fedx_2d[fedx_2d['ACTUAL_PICKUP_DATE'] > date_after].copy()

#Loading Excel Sheets
fedx_2d_rates_fy23 = pd.read_excel('fedex_costs.xlsx', sheet_name = '2d_rates_fy23').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")
fedx_2d_rates_fy24 = pd.read_excel('fedex_costs.xlsx', sheet_name = '2d_rates_fy24').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")
fedx_2d_rates_fy25 = pd.read_excel('fedex_costs.xlsx', sheet_name = '2d_rates_fy25').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")

fedx_2d['BILLED_WEIGHT'] = fedx_2d['BILLED_WEIGHT'].astype('float')
fedx_2d['TOTAL_FREIGHT_AMOUNT'] = fedx_2d['TOTAL_FREIGHT_AMOUNT'].astype('float')
fedx_2d_rates_fy23['FreightPrice'] = fedx_2d_rates_fy23['FreightPrice'].astype('float')
fedx_2d_rates_fy23['WEIGHT'] = fedx_2d_rates_fy23['WEIGHT'].astype('float32')
fedx_2d_rates_fy24['FreightPrice'] = fedx_2d_rates_fy24['FreightPrice'].astype('float')
fedx_2d_rates_fy24['WEIGHT'] = fedx_2d_rates_fy24['WEIGHT'].astype('float')
fedx_2d_rates_fy25['FreightPrice'] = fedx_2d_rates_fy25['FreightPrice'].astype('float')
fedx_2d_rates_fy25['WEIGHT'] = fedx_2d_rates_fy25['WEIGHT'].astype('float')

#Merging Sheets based on Pick-up Dates
fedx_2d_zoned_fy23 = pd.merge(fedx_2d_fy23, fedx_2d_rates_fy23, 
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")
fedx_2d_zoned_fy24 = pd.merge(fedx_2d_fy24, fedx_2d_rates_fy24,
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")
fedx_2d_zoned_fy25 = pd.merge(fedx_2d_fy25, fedx_2d_rates_fy25,
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")

fedx_2d_zoned_fy = pd.concat([fedx_2d_zoned_fy23, fedx_2d_zoned_fy24, fedx_2d_zoned_fy25], ignore_index = True)

#Seperating data where merge was sucessful and unsucessful
fedx_2d_zoned0 = fedx_2d_zoned_fy[fedx_2d_zoned_fy['ZONE'].notnull()].copy()
fedx_2d_missing1 = fedx_2d_zoned_fy[fedx_2d_zoned_fy['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

#Merging table with missing zones to all zone tables, than concat back to successful zones
fedx_2d_retry1 = fedx_2d_missing1.merge(fedx_2d_rates_fy25, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")
fedx_2d_zoned1 = fedx_2d_retry1[fedx_2d_retry1['ZONE'].notnull()].copy()
fedx_2d_missing2 = fedx_2d_retry1[fedx_2d_retry1['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

fedx_2d_retry2 = fedx_2d_missing2.merge(fedx_2d_rates_fy24, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")
fedx_2d_zoned2 = fedx_2d_retry2[fedx_2d_retry2['ZONE'].notnull()].copy()
fedx_2d_missing3 = fedx_2d_retry2[fedx_2d_retry2['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

fedx_2d_retry3 = fedx_2d_missing3.merge(fedx_2d_rates_fy23, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")

fedx_2d_zoned = pd.concat([fedx_2d_zoned0, fedx_2d_zoned1, fedx_2d_zoned2, fedx_2d_retry3], ignore_index = True)

#Deleting Uneeded Variables to Release Memory
del fedx_2d, fedx_2d_fy23, fedx_2d_fy24, fedx_2d_fy25, fedx_2d_rates_fy23, fedx_2d_rates_fy24, fedx_2d_rates_fy25
del fedx_2d_zoned_fy23, fedx_2d_zoned_fy24, fedx_2d_zoned_fy25, fedx_2d_zoned_fy, fedx_2d_zoned0, fedx_2d_zoned1
del fedx_2d_zoned2, fedx_2d_retry1, fedx_2d_retry2, fedx_2d_retry3, fedx_2d_missing1, fedx_2d_missing2, fedx_2d_missing3

print(f'FEDEX 2Day Merge Results')
print(f'Missing Zones: ',fedx_2d_zoned["ZONE"].isna().sum())
print(f'Total Rows: ', len(fedx_2d_zoned))
print(round(((fedx_2d_zoned["ZONE"].isna().sum() / len(fedx_2d_zoned))*100),2), f'% Zones Missing')

In [ ]:
#FEDEX First Overnight
fedx_fo = fedx[fedx['SERVICE_LVL_CD_DESC'] == 'FedEx First Overnight'].copy()

#Splitting data by Pick-up dates: Before 10/10/23, 10/10/23-10/10/24, After 10/10/24
date_before = pd.Timestamp('2023-10-10')
date_after = pd.Timestamp('2024-10-10')

fedx_fo_fy23 = fedx_fo[fedx_fo['ACTUAL_PICKUP_DATE'] < date_before].copy()
fedx_fo_fy24 = fedx_fo[(fedx_fo['ACTUAL_PICKUP_DATE'] >= date_before) & (fedx_fo['ACTUAL_PICKUP_DATE'] <= date_after)].copy()
fedx_fo_fy25 = fedx_fo[fedx_fo['ACTUAL_PICKUP_DATE'] > date_after].copy()

#Loading Excel Sheets
fedx_fo_rates_fy23 = pd.read_excel('fedex_costs.xlsx', sheet_name = 'fo_rates_fy23').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")
fedx_fo_rates_fy24 = pd.read_excel('fedex_costs.xlsx', sheet_name = 'fo_rates_fy24').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")
fedx_fo_rates_fy25 = pd.read_excel('fedex_costs.xlsx', sheet_name = 'fo_rates_fy25').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")

fedx_fo['BILLED_WEIGHT'] = fedx_fo['BILLED_WEIGHT'].astype('float')
fedx_fo['TOTAL_FREIGHT_AMOUNT'] = fedx_fo['TOTAL_FREIGHT_AMOUNT'].astype('float')
fedx_fo_rates_fy23['FreightPrice'] = fedx_fo_rates_fy23['FreightPrice'].astype('float')
fedx_fo_rates_fy23['WEIGHT'] = fedx_fo_rates_fy23['WEIGHT'].astype('float32')
fedx_fo_rates_fy24['FreightPrice'] = fedx_fo_rates_fy24['FreightPrice'].astype('float')
fedx_fo_rates_fy24['WEIGHT'] = fedx_fo_rates_fy24['WEIGHT'].astype('float')
fedx_fo_rates_fy25['FreightPrice'] = fedx_fo_rates_fy25['FreightPrice'].astype('float')
fedx_fo_rates_fy25['WEIGHT'] = fedx_fo_rates_fy25['WEIGHT'].astype('float')

#Merging Sheets based on Pick-up Dates
fedx_fo_zoned_fy23 = pd.merge(fedx_fo_fy23, fedx_fo_rates_fy23, 
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")
fedx_fo_zoned_fy24 = pd.merge(fedx_fo_fy24, fedx_fo_rates_fy24,
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")
fedx_fo_zoned_fy25 = pd.merge(fedx_fo_fy25, fedx_fo_rates_fy25,
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")

fedx_fo_zoned_fy = pd.concat([fedx_fo_zoned_fy23, fedx_fo_zoned_fy24, fedx_fo_zoned_fy25], ignore_index = True)

#Seperating data where merge was sucessful and unsucessful
fedx_fo_zoned0 = fedx_fo_zoned_fy[fedx_fo_zoned_fy['ZONE'].notnull()].copy()
fedx_fo_missing1 = fedx_fo_zoned_fy[fedx_fo_zoned_fy['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

#Merging table with missing zones to all zone tables, than concat back to successful zones
fedx_fo_retry1 = fedx_fo_missing1.merge(fedx_fo_rates_fy25, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")
fedx_fo_zoned1 = fedx_fo_retry1[fedx_fo_retry1['ZONE'].notnull()].copy()
fedx_fo_missing2 = fedx_fo_retry1[fedx_fo_retry1['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

fedx_fo_retry2 = fedx_fo_missing2.merge(fedx_fo_rates_fy24, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")
fedx_fo_zoned2 = fedx_fo_retry2[fedx_fo_retry2['ZONE'].notnull()].copy()
fedx_fo_missing3 = fedx_fo_retry2[fedx_fo_retry2['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

fedx_fo_retry3 = fedx_fo_missing3.merge(fedx_fo_rates_fy23, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")

fedx_fo_zoned = pd.concat([fedx_fo_zoned0, fedx_fo_zoned1, fedx_fo_zoned2, fedx_fo_retry3], ignore_index = True)

#Deleting Uneeded Variables to Release Memory
del fedx_fo, fedx_fo_fy23, fedx_fo_fy24, fedx_fo_fy25, fedx_fo_rates_fy23, fedx_fo_rates_fy24, fedx_fo_rates_fy25
del fedx_fo_zoned_fy23, fedx_fo_zoned_fy24, fedx_fo_zoned_fy25, fedx_fo_zoned_fy, fedx_fo_zoned0, fedx_fo_zoned1
del fedx_fo_zoned2, fedx_fo_retry1, fedx_fo_retry2, fedx_fo_retry3, fedx_fo_missing1, fedx_fo_missing2, fedx_fo_missing3

print(f'FEDEX First Overnight Merge Results')
print(f'Missing Zones: ',fedx_fo_zoned["ZONE"].isna().sum())
print(f'Total Rows: ', len(fedx_fo_zoned))
print(round(((fedx_fo_zoned["ZONE"].isna().sum() / len(fedx_fo_zoned))*100),2), f'% Zones Missing')

In [ ]:
#FEDEX 2Day AM
fedx_2dam = fedx[fedx['SERVICE_LVL_CD_DESC'] == 'FedEx 2Day AM'].copy()

#Splitting data by Pick-up dates: Be2damre 10/10/23, 10/10/23-10/10/24, After 10/10/24
date_be2damre = pd.Timestamp('2023-10-10')
date_after = pd.Timestamp('2024-10-10')

fedx_2dam_fy23 = fedx_2dam[fedx_2dam['ACTUAL_PICKUP_DATE'] < date_be2damre].copy()
fedx_2dam_fy24 = fedx_2dam[(fedx_2dam['ACTUAL_PICKUP_DATE'] >= date_be2damre) & (fedx_2dam['ACTUAL_PICKUP_DATE'] <= date_after)].copy()
fedx_2dam_fy25 = fedx_2dam[fedx_2dam['ACTUAL_PICKUP_DATE'] > date_after].copy()

#Loading Excel Sheets
fedx_2dam_rates_fy23 = pd.read_excel('fedex_costs.xlsx', sheet_name = '2dam_rates_fy23').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")
fedx_2dam_rates_fy24 = pd.read_excel('fedex_costs.xlsx', sheet_name = '2dam_rates_fy24').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")
fedx_2dam_rates_fy25 = pd.read_excel('fedex_costs.xlsx', sheet_name = '2dam_rates_fy25').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")

fedx_2dam['BILLED_WEIGHT'] = fedx_2dam['BILLED_WEIGHT'].astype('float')
fedx_2dam['TOTAL_FREIGHT_AMOUNT'] = fedx_2dam['TOTAL_FREIGHT_AMOUNT'].astype('float')
fedx_2dam_rates_fy23['FreightPrice'] = fedx_2dam_rates_fy23['FreightPrice'].astype('float')
fedx_2dam_rates_fy23['WEIGHT'] = fedx_2dam_rates_fy23['WEIGHT'].astype('float32')
fedx_2dam_rates_fy24['FreightPrice'] = fedx_2dam_rates_fy24['FreightPrice'].astype('float')
fedx_2dam_rates_fy24['WEIGHT'] = fedx_2dam_rates_fy24['WEIGHT'].astype('float')
fedx_2dam_rates_fy25['FreightPrice'] = fedx_2dam_rates_fy25['FreightPrice'].astype('float')
fedx_2dam_rates_fy25['WEIGHT'] = fedx_2dam_rates_fy25['WEIGHT'].astype('float')

#Merging Sheets based on Pick-up Dates
fedx_2dam_zoned_fy23 = pd.merge(fedx_2dam_fy23, fedx_2dam_rates_fy23, 
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")
fedx_2dam_zoned_fy24 = pd.merge(fedx_2dam_fy24, fedx_2dam_rates_fy24,
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")
fedx_2dam_zoned_fy25 = pd.merge(fedx_2dam_fy25, fedx_2dam_rates_fy25,
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")

fedx_2dam_zoned_fy = pd.concat([fedx_2dam_zoned_fy23, fedx_2dam_zoned_fy24, fedx_2dam_zoned_fy25], ignore_index = True)

#Seperating data where merge was sucessful and unsucessful
fedx_2dam_zoned0 = fedx_2dam_zoned_fy[fedx_2dam_zoned_fy['ZONE'].notnull()].copy()
fedx_2dam_missing1 = fedx_2dam_zoned_fy[fedx_2dam_zoned_fy['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

#Merging table with missing zones to all zone tables, than concat back to successful zones
fedx_2dam_retry1 = fedx_2dam_missing1.merge(fedx_2dam_rates_fy25, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")
fedx_2dam_zoned1 = fedx_2dam_retry1[fedx_2dam_retry1['ZONE'].notnull()].copy()
fedx_2dam_missing2 = fedx_2dam_retry1[fedx_2dam_retry1['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

fedx_2dam_retry2 = fedx_2dam_missing2.merge(fedx_2dam_rates_fy24, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")
fedx_2dam_zoned2 = fedx_2dam_retry2[fedx_2dam_retry2['ZONE'].notnull()].copy()
fedx_2dam_missing3 = fedx_2dam_retry2[fedx_2dam_retry2['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

fedx_2dam_retry3 = fedx_2dam_missing3.merge(fedx_2dam_rates_fy23, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")

fedx_2dam_zoned = pd.concat([fedx_2dam_zoned0, fedx_2dam_zoned1, fedx_2dam_zoned2, fedx_2dam_retry3], ignore_index = True)

#Deleting Uneeded Variables to Release Memory
del fedx_2dam, fedx_2dam_fy23, fedx_2dam_fy24, fedx_2dam_fy25, fedx_2dam_rates_fy23, fedx_2dam_rates_fy24, fedx_2dam_rates_fy25
del fedx_2dam_zoned_fy23, fedx_2dam_zoned_fy24, fedx_2dam_zoned_fy25, fedx_2dam_zoned_fy, fedx_2dam_zoned0, fedx_2dam_zoned1
del fedx_2dam_zoned2, fedx_2dam_retry1, fedx_2dam_retry2, fedx_2dam_retry3, fedx_2dam_missing1, fedx_2dam_missing2, fedx_2dam_missing3

print(f'FEDEX 2Day AM Merge Results')
print(f'Missing Zones: ',fedx_2dam_zoned["ZONE"].isna().sum())
print(f'Total Rows: ', len(fedx_2dam_zoned))
print(round(((fedx_2dam_zoned["ZONE"].isna().sum() / len(fedx_2dam_zoned))*100),2), f'% Zones Missing')

In [ ]:
#FedEx Express Saver
fedx_es = fedx[fedx['SERVICE_LVL_CD_DESC'] == 'FedEx Express Saver'].copy()

#Splitting data by Pick-up dates: Beesre 10/10/23, 10/10/23-10/10/24, After 10/10/24
date_beesre = pd.Timestamp('2023-10-10')
date_after = pd.Timestamp('2024-10-10')

fedx_es_fy23 = fedx_es[fedx_es['ACTUAL_PICKUP_DATE'] < date_beesre].copy()
fedx_es_fy24 = fedx_es[(fedx_es['ACTUAL_PICKUP_DATE'] >= date_beesre) & (fedx_es['ACTUAL_PICKUP_DATE'] <= date_after)].copy()
fedx_es_fy25 = fedx_es[fedx_es['ACTUAL_PICKUP_DATE'] > date_after].copy()

#Loading Excel Sheets
fedx_es_rates_fy23 = pd.read_excel('fedex_costs.xlsx', sheet_name = 'es_rates_fy23').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")
fedx_es_rates_fy24 = pd.read_excel('fedex_costs.xlsx', sheet_name = 'es_rates_fy24').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")
fedx_es_rates_fy25 = pd.read_excel('fedex_costs.xlsx', sheet_name = 'es_rates_fy25').melt(id_vars="WEIGHT", var_name="ZONE", value_name="FreightPrice")

fedx_es['BILLED_WEIGHT'] = fedx_es['BILLED_WEIGHT'].astype('float')
fedx_es['TOTAL_FREIGHT_AMOUNT'] = fedx_es['TOTAL_FREIGHT_AMOUNT'].astype('float')
fedx_es_rates_fy23['FreightPrice'] = fedx_es_rates_fy23['FreightPrice'].astype('float')
fedx_es_rates_fy23['WEIGHT'] = fedx_es_rates_fy23['WEIGHT'].astype('float32')
fedx_es_rates_fy24['FreightPrice'] = fedx_es_rates_fy24['FreightPrice'].astype('float')
fedx_es_rates_fy24['WEIGHT'] = fedx_es_rates_fy24['WEIGHT'].astype('float')
fedx_es_rates_fy25['FreightPrice'] = fedx_es_rates_fy25['FreightPrice'].astype('float')
fedx_es_rates_fy25['WEIGHT'] = fedx_es_rates_fy25['WEIGHT'].astype('float')

#Merging Sheets based on Pick-up Dates
fedx_es_zoned_fy23 = pd.merge(fedx_es_fy23, fedx_es_rates_fy23, 
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")
fedx_es_zoned_fy24 = pd.merge(fedx_es_fy24, fedx_es_rates_fy24,
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")
fedx_es_zoned_fy25 = pd.merge(fedx_es_fy25, fedx_es_rates_fy25,
                                  left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], right_on = ["WEIGHT", "FreightPrice"], how = "left")

fedx_es_zoned_fy = pd.concat([fedx_es_zoned_fy23, fedx_es_zoned_fy24, fedx_es_zoned_fy25], ignore_index = True)

#Seperating data where merge was sucessful and unsucessful
fedx_es_zoned0 = fedx_es_zoned_fy[fedx_es_zoned_fy['ZONE'].notnull()].copy()
fedx_es_missing1 = fedx_es_zoned_fy[fedx_es_zoned_fy['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

#Merging table with missing zones to all zone tables, than concat back to successful zones
fedx_es_retry1 = fedx_es_missing1.merge(fedx_es_rates_fy25, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")
fedx_es_zoned1 = fedx_es_retry1[fedx_es_retry1['ZONE'].notnull()].copy()
fedx_es_missing2 = fedx_es_retry1[fedx_es_retry1['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

fedx_es_retry2 = fedx_es_missing2.merge(fedx_es_rates_fy24, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")
fedx_es_zoned2 = fedx_es_retry2[fedx_es_retry2['ZONE'].notnull()].copy()
fedx_es_missing3 = fedx_es_retry2[fedx_es_retry2['ZONE'].isnull()].drop(columns = ["WEIGHT", "FreightPrice", "ZONE"]).copy()

fedx_es_retry3 = fedx_es_missing3.merge(fedx_es_rates_fy23, left_on = ["BILLED_WEIGHT", "TOTAL_FREIGHT_AMOUNT"], 
                                              right_on = ["WEIGHT", "FreightPrice"], how = "left")

fedx_es_zoned = pd.concat([fedx_es_zoned0, fedx_es_zoned1, fedx_es_zoned2, fedx_es_retry3], ignore_index = True)

#Deleting Uneeded Variables to Release Memory
del fedx_es, fedx_es_fy23, fedx_es_fy24, fedx_es_fy25, fedx_es_rates_fy23, fedx_es_rates_fy24, fedx_es_rates_fy25
del fedx_es_zoned_fy23, fedx_es_zoned_fy24, fedx_es_zoned_fy25, fedx_es_zoned_fy, fedx_es_zoned0, fedx_es_zoned1
del fedx_es_zoned2, fedx_es_retry1, fedx_es_retry2, fedx_es_retry3, fedx_es_missing1, fedx_es_missing2, fedx_es_missing3

print(f'FedEx Express Saver Merge Results')
print(f'Missing Zones: ',fedx_es_zoned["ZONE"].isna().sum())
print(f'Total Rows: ', len(fedx_es_zoned))
print(round(((fedx_es_zoned["ZONE"].isna().sum() / len(fedx_es_zoned))*100),2), f'% Zones Missing')

In [ ]:
#FedEx Economy
fedx_eco = fedx[fedx['SERVICE_LVL_CD_DESC'] == 'FedEx Economy']

In [ ]:
#Append all FedEx
fedx_dfs = [fedx_hd_zoned, fedx_po_zoned, fedx_so_zoned, fedx_ground_zoned, fedx_2d_zoned, fedx_fo_zoned, fedx_2dam_zoned, fedx_es_zoned, fedx_eco]
fedx_zoned = pd.concat(fedx_dfs, ignore_index = True)

print(f'FedEx Merge Results')
print(f'Missing Zones: ',fedx_zoned["ZONE"].isna().sum())
print(f'Total Rows: ', len(fedx_zoned))
print(round(((fedx_zoned["ZONE"].isna().sum() / len(fedx_zoned))*100),2), f'% Zones Missing')

del fedx_hd_zoned, fedx_po_zoned, fedx_so_zoned, fedx_ground_zoned, fedx_2d_zoned, fedx_fo_zoned, fedx_2dam_zoned, fedx_es_zoned, fedx_eco

## Zone Assignments - ZIP Code Matching

In [ ]:
#Recombine FEDEX and UPS Data
data = pd.concat([fedx_zoned, upso_zoned], ignore_index = True).drop(columns = (["WEIGHT", "FreightPrice"]))
data = pd.concat([data, data_zoned], ignore_index = True)
data.info()

In [ ]:
#Recombine FEDEX and UPS Data
#data = pd.concat([fedx_zoned, upso_zoned], ignore_index = True).drop(columns = (["WEIGHT", "FreightPrice"]))
#data = pd.concat([data, data_zoned], ignore_index = True)

#Seperating data where merge was sucessful and unsucessful
data_zoned = data[data['ZONE'].notnull()].copy()
data_missing = data[data['ZONE'].isnull()].drop(columns = ["ZONE"]).copy()

#Creating table with unique ZIP/Zone Combinations
zip_zone = data[['SHIPPER_ZIP', 'RECIPIENT_ZIP', 'ZONE']].drop_duplicates(subset=['SHIPPER_ZIP', 'RECIPIENT_ZIP']).dropna().reset_index(drop=True)

#Merging unmached zone data matched by price with unique ZIP/Zone table
data_zip_matched = pd.merge(data_missing, zip_zone, on = ['SHIPPER_ZIP', 'RECIPIENT_ZIP'], how = 'left')

#Merging with Second ZIP-Zone Table (From CMOP Data)
zips = pd.read_excel("ZIPs.xlsx", sheet_name = "ZIPS")
zips[['SHIPPER_ZIP', 'RECIPIENT_ZIP', 'ZONE']] = zips[['SHIPPER_ZIP', 'RECIPIENT_ZIP', 'ZONE']].astype('object')
data_zip_matched_1 = data_zip_matched[data_zip_matched['ZONE'].notnull()]
data_missing_1 = data_zip_matched[data_zip_matched['ZONE'].isnull()].drop(columns = ["ZONE"]).copy()
data_zip_matched_2 = pd.merge(data_missing_1, zips, on = ['SHIPPER_ZIP', 'RECIPIENT_ZIP'], how = 'left')

#Merging data matched by price with data matched by ZIP
data = pd.concat([data_zoned, data_zip_matched_1, data_zip_matched_2], ignore_index = True)

del zips, zip_zone

## Zone Assignments - Final Results

In [ ]:
result = data.groupby('SERVICE_LVL_CD_DESC').apply(
    lambda group: pd.Series({
        'Counts': len(group),
        '# of Missing ZONEs ': group['ZONE'].isna().sum(),
        '% Zones Missing': round(((group['ZONE'].isna().mean()) * 100),2)})).reset_index()
result

# CMOP VS. TRANSCOM Classification

In [ ]:
#Import CMOP Account Data
cmop_accounts = pd.read_excel('Transcom CMOP Account Data.xlsx', sheet_name = 'Sheet1')
cmop_list = cmop_accounts['TRANSCOM Account Number'].tolist()

#Creating a new TRANSCOM/CMOP Column
data['TRANSCOM/CMOP'] = data['FREIGHT_PAYEE_ACCOUNT_NUMBER'].isin(cmop_list).map({True: 'CMOP', False: 'TRANSCOM'})
del cmop_accounts

# Final Cleaning & Export

In [ ]:
#Rearrange Columns
data = data[['REPORT_MONTH', 'SHIPPING_AGENCY_LVL_3', 'CHILD_NAME', 'CARRIER_SCAC', 'SHIPMENT_TRACKING_NUMBER',
 'SHIPMENT_TRACKING_GROUP_ID', 'FREIGHT_PAYEE_ACCOUNT_NUMBER', 'LEAD_TRACKING_NUMBER', 'RESIDENTIAL_FLAG',
 'RATE_SERVICE_FLAG', 'DELIVERY_STATUS_CODE', 'DELIVERY_STATUS_FLAG', 'DELAYS', 'SERVICE_LVL_CD_DESC', 'PACKAGE_TYPE',
 'ACTUAL_PICKUP_DATE', 'ACTUAL_DELIVERY_DATE', 'CONTRACT_DELIVERY_DATE', 'INVOICE_DATE', 'SHIPPER_CITY_NAME',
 'SHIPPER_STATE_PRVNC', 'SHIPPER_ZIP', 'RECIPIENT_CITY_NAME', 'RECIPIENT_STATE_PRVNC', 'RECIPIENT_ZIP', 'ZONE','NUM_OF_SHIPMENTS',
 'NUM_OF_PACKAGES', 'PACKAGE_LENGTH', 'PACKAGE_HEIGHT', 'PACKAGE_WIDTH', 'BILLED_WEIGHT', 'ACTUAL_WEIGHT',
 'TOTAL_SURCHARGE_AMOUNT', 'TOTAL_FREIGHT_AMOUNT', 'TOTAL_DISCOUNT_AMOUNT', 'TOTAL_NET_AMOUNT']]

#Renaming columns
cols_rn = {
 'REPORT_MONTH':"Report Month", 
 'SHIPPING_AGENCY_LVL_3':'Shipping Agency Lvl 3', 
 'CHILD_NAME':'Child Name', 
 'CARRIER_SCAC':'Carrier', 
 'SHIPMENT_TRACKING_NUMBER':'Shipping Tracking #', 
 'SHIPMENT_TRACKING_GROUP_ID':"Shipment Tracking Group ID",
 'FREIGHT_PAYEE_ACCOUNT_NUMBER':'Freight Payee Account Number',
 'LEAD_TRACKING_NUMBER':'Lead Tracking #',
 'RESIDENTIAL_FLAG':'Residential Flag',
 'RATE_SERVICE_FLAG':'Rate Service Flag',
 'DELIVERY_STATUS_CODE':'Delivery Status Code',
 'DELIVERY_STATUS_FLAG':'Delivery Status Flag',
 'DELAYS':'Delays', 
 'SERVICE_LVL_CD_DESC':'Service Level',
 'PACKAGE_TYPE':'Package Type',
 'ACTUAL_PICKUP_DATE':'Actual Pick-up Date',
 'ACTUAL_DELIVERY_DATE':'Actual Delivery Date',
 'CONTRACT_DELIVERY_DATE':'Contract Delivery Date', 
 'INVOICE_DATE':'Invoice Date', 'SHIPPER_CITY_NAME': 'Shipper City',
 'SHIPPER_STATE_PRVNC': 'Shipper State', 'SHIPPER_ZIP':'Shipper Zip', 'RECIPIENT_CITY_NAME': 'Recipient City',
 'RECIPIENT_STATE_PRVNC': 'Recipient State', 'RECIPIENT_ZIP':'Recipient Zip', 'NUM_OF_SHIPMENTS': '# of Shipments', 
 'NUM_OF_PACKAGES': '# of Packages', 'PACKAGE_LENGTH':'Length', 'PACKAGE_HEIGHT':'Height', 'PACKAGE_WIDTH':'Width',
 'BILLED_WEIGHT':'Billed Weight', 'ACTUAL_WEIGHT':'Actual Weight', 'TOTAL_SURCHARGE_AMOUNT':'Surcharge Amount',
 'TOTAL_FREIGHT_AMOUNT':'Freight Amount', 'TOTAL_DISCOUNT_AMOUNT':'Discount Amount','TOTAL_NET_AMOUNT':'Net Amount', 'ZONE':'Zone'
}
data = data.rename(columns = cols_rn)

In [ ]:
#Export Data
output_path = os.path.join(output_dir, output_filename)  # Save to Output folder
data.to_csv(output_path, index=False)